In [ ]:
%matplotlib inline
import inspect, os, sys, copy, pytz, re, glob, random, praw, csv
import simplejson as json
import pandas as pd
from dateutil import parser
import datetime
import matplotlib.pyplot as plt   # Matplotlib for plotting
import matplotlib.dates as md
import numpy as np
import seaborn as sns
from collections import Counter, defaultdict
import re, urllib

from pathlib import Path
from datetime import datetime
import logging

utc=pytz.UTC

ENV = "production"
os.environ['CS_ENV'] = 'production'
BASE_DIR = "/home/nathan/CivilServant"
FILE_BASE_DIR="/home/nathan/reddit_archive/"
sys.path.append(BASE_DIR)
subreddit_id = "2qr7i"

LOG_PATH = str(Path(BASE_DIR, "logs", "praw_messenger_%s.log" % ENV))
logging.basicConfig(filename=LOG_PATH, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(sys.stdout))

# ARCHIVE_DIR = "/home/nathan/reddit_archive/03.2017"

# with open(os.path.join(BASE_DIR, "config") + "/{env}.json".format(env=ENV), "r") as config:
#   DBCONFIG = json.loads(config.read())

# ### LOAD SQLALCHEMY
# from sqlalchemy import create_engine
# from sqlalchemy.orm import sessionmaker
# from sqlalchemy import text, and_, or_
# from app.models import Base, SubredditPage, FrontPage, Subreddit, Post, ModAction, Experiment
# from utils.common import PageType

# db_engine = create_engine("mysql://{user}:{password}@{host}/{database}".format(
#     host = DBCONFIG['host'],
#     user = DBCONFIG['user'],
#     password = DBCONFIG['password'],
#     database = DBCONFIG['database']))
# DBSession = sessionmaker(bind=db_engine)
# db_session = DBSession()

### LOAD PRAW
import reddit.connection
conn = reddit.connection.Connect(base_dir=BASE_DIR)
r = conn.connect(use_db_keys=False)

#notebook_dir = os.getcwd()
#os.chdir(notebook_dir)
#import praw.errors


### FILTER OUT DEPRECATION WARNINGS ASSOCIATED WITH DECORATORS
# https://github.com/ipython/ipython/issues/9242
#import warnings
#warnings.filterwarnings('ignore', category=DeprecationWarning, message='.*use @default decorator instead.*')

# Get list of study participants
We avoid archiving these accounts for privacy reasons

# Construct and personalize the message

## Construct the message template

In [ ]:
MESSAGE_SUBJECT = "Welcome! Help r/feminism understand first-time commenters with 3 questions"
MESSAGE_TEMPLATE = """Dear {username},

Hello and welcome from r/feminism! 

Might you answer a 2 minute, 3-question survey to help us learn more about your experience? We're' currently working with researchers at Princeton University to improve newcomer experiences.

[Learn more and answer the survey here]({url}). Thanks! 

-- [CivilServant](http://civilservant.io/) and the [r/feminism moderators](https://www.reddit.com/r/feminism/about/moderators)
"""

In [ ]:
print(MESSAGE_TEMPLATE.format(username="natematias", 
                              url="https://docs.google.com/forms/d/e/1FAIpQLSc1Atz_YmZVZIBILyVRsnZYHT3KrGqEseM-oUyz6dLllEfcVg/viewform?usp=pp_url&entry.360321409={username}".format(
                                  username="natematias")))

In [ ]:
# r.send_message("natematias", MESSAGE_SUBJECT, 
#                MESSAGE_TEMPLATE.format(username="natematias",
#                                        url="https://docs.google.com/forms/d/e/1FAIpQLSc1Atz_YmZVZIBILyVRsnZYHT3KrGqEseM-oUyz6dLllEfcVg/viewform?usp=pp_url&entry.360321409={username}".format(
#                                             username="natematias")))

## Add the ability to validate users

In [ ]:
def get_reddit_user(username):
    try:
        user_dict = r.get_redditor(username, fetch=True).json_dict
        logging.info("User %s found: %s" %(username, str(user_dict)))
        return user_dict
    except praw.errors.NotFound as e:
        logging.error("User not found: %s" % username)
    except Exception as e:
        logging.exception("Failed to retrieve user")

In [ ]:
import time
send_log_dir = "/home/nathan/CivilServant-Analysis/SOC412/feminism"
send_log_filename = "followup_survey_experiment_12.csv"
def sent_message_previously(username):
    with open(os.path.join(send_log_dir, send_log_filename), "r") as f:
        send_log = []
        for row in csv.DictReader(f):
            send_log.append(row)
            if(username == row['username']):
                return True
    return False

## Add the ability to send messages

In [ ]:
## RETURN TRUE IF MESSAGE WAS SENT SUCCESSFULLY
def send_reddit_message(**message_data):
    user = message_data["username"]    
    message = MESSAGE_TEMPLATE.format(**message_data)    
    try:
        logging.info("Sending a message to user %s with data %s" % (user, str(message_data)))
        response = r.send_message(user, MESSAGE_SUBJECT, message, raise_captcha_exception=True)
        if response["errors"]:
            logging.error("Error in response when sending a message to user %s: %s" % (user, str(response)))
            return False
        else:
            logging.info("Message successfully sent to user %s" % user)
            return True
    except praw.errors.InvalidCaptcha as e:
        logging.exception("Message sending requires a captcha")
        logging.error(e.response)
        return False
    except Exception as e:
        logging.exception("Failed to send message to %s" % user)
        return False

## Wrap the messenger function with anything survey-specific

In [ ]:
def send_survey_message(**message_data):
    message = MESSAGE_TEMPLATE.format(**message_data)        
    sent_status = send_reddit_message(**message_data)
    with open(os.path.join(send_log_dir,send_log_filename), "a") as f:
        print("APPENDING TO FILE: {0}".format(message_data))
        f.write(",".join(map(str, [
            message_data['username'],
            True, #account exists
            message_data['get_info_date'],
            message_data['deleted'],
            message_data['url'],
            sent_status
        ])) + "\n")
        f.close()
    
    logging.info("url: %s" % message_data["url"])
    logging.info("get_info_data: %s" % message_data["get_info_date"])
    logging.info("deleted: %s" % message_data["deleted"])
    

In [ ]:
def decide_and_send_to_user(username):
    if(sent_message_previously(username)):
        logging.info("User %s previously received a message" % username)
        return None
    else:
        time.sleep(2)
        user = get_reddit_user(username)

    if user:
        send_survey_message(
            username = username,
            get_info_date = str(datetime.utcnow()),
            deleted = False,
            url="https://docs.google.com/forms/d/e/1FAIpQLSc1Atz_YmZVZIBILyVRsnZYHT3KrGqEseM-oUyz6dLllEfcVg/viewform?usp=pp_url&entry.360321409={username}".format(
                username=username))
    else:
        with open(os.path.join(send_log_dir, send_log_filename), "a") as f:
            f.write(",".join(map(str, [
                username,
                False,
                str(datetime.utcnow()),
                None,
                None,
                False
            ])) + "\n")
            f.close()

# Load Survey Recipients

In [ ]:
## LOAD RECIPIENTS FROM A CSV FILE
## THEN USE THE CODE BELOW TO SEND THEM THINGS
filename = "experiment_12_eligible_screen_names-08.02.2018.csv"
eligible_usernames = []
with open(os.path.join("..","inputs", filename), "r") as f:
    for row in csv.DictReader(f):
        eligible_usernames.append(row['screen_name'])
print("Found {0} eligible usernames".format(len(eligible_usernames)))

# Send Surveys to Participants

In [ ]:
for username in eligible_usernames:
    decide_and_send_to_user(username)